In [1]:
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
ls

IMDB-Movie-Data.csv      ml-latest/               the-movies-dataset/
imdb.csv*                ml-latest-small/         tmdb-5000-movie-dataset/
ml-100k/                 movie_analysis.ipynb     untitled.txt
ml-1m/                   notebook.ipynb
ml-20m/                  regression.ipynb


In [3]:
cd the-movies-dataset/

/Users/yoshidatomoya/Documents/GCI/movie_analysis/the-movies-dataset


In [4]:
ls

actors.csv           jobs.csv             links_small.csv*
credits.csv*         keywords.csv*        movies_metadata.csv*
directors.csv        links.csv*           ratings_small.csv*


In [5]:
cd ..

/Users/yoshidatomoya/Documents/GCI/movie_analysis


In [27]:
metadata_URL = "./the-movies-dataset/movies_metadata.csv"
genres_URL = "./the-movies-dataset/genres.csv"
scores_URL = "./ml-latest/95%_least_plausible.csv"
links_URL = "./the-movies-dataset/links.csv"
links_small_URL = "./the-movies-dataset/links_small.csv"
credits_URL = "./the-movies-dataset/credits.csv"
directors_URL = "./the-movies-dataset/directors.csv"
actors_URL = "./the-movies-dataset/actors.csv"
jobs_URL = "./the-movies-dataset/jobs.csv"

In [7]:
# データフレーム中のstringをdict型のリストに変換する
def make_list_from_str(string):
    if isinstance(string, str):
        result_list = eval(string)
    else:
        result_list = np.nan
    return result_list

In [8]:
def extract_ids(genres_list):
    id_list = []
    for genre in genres_list:
        id_list.append(genre["id"])
    return id_list

In [9]:
def extract_names(genres_list):
    id_list = []
    for genre in genres_list:
        id_list.append(genre["name"])
    return id_list

In [10]:
metadata_df = pd.read_csv(metadata_URL)[["id", "genres"]]
metadata_df["genres"] = metadata_df["genres"].map(make_list_from_str)
metadata_df["genres"] = metadata_df["genres"].map(extract_names)
metadata_df["genre_count"] = metadata_df["genres"].map(lambda x: len(x))
metadata_df.head()

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,genres,genre_count
0,862,"[Animation, Comedy, Family]",3
1,8844,"[Adventure, Fantasy, Family]",3
2,15602,"[Romance, Comedy]",2
3,31357,"[Comedy, Drama, Romance]",3
4,11862,[Comedy],1


In [13]:
##すべてのジャンルのidリスト、nameリストを作成

genres_df = pd.read_csv(genres_URL)
all_genre_id = genres_df["id"].tolist()
all_genre_name = genres_df["name"].tolist()
print("ids: ", all_genre_id)
print("names: ", all_genre_name)

ids:  [16, 35, 10751, 12, 14, 10749, 18, 28, 80, 53, 27, 36, 878, 9648, 10752, 10769, 10402, 99, 37, 10770]
names:  ['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie']


In [14]:
metadata_df.groupby("genre_count")["id"].count()

genre_count
0     2442
1    14559
2    14480
3     9586
4     3380
5      835
6      157
7       24
8        3
Name: id, dtype: int64

In [15]:
metadata_df = metadata_df[metadata_df["genre_count"] > 0]
metadata_df.groupby("genre_count")["id"].count()

genre_count
1    14559
2    14480
3     9586
4     3380
5      835
6      157
7       24
8        3
Name: id, dtype: int64

In [16]:
genre_dummies = pd.get_dummies(metadata_df["genres"].apply(pd.Series).stack()).sum(level=0)
genre_dummies.head()

,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,Drama,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
genre_dummies = genre_dummies[all_genre_name]
genre_dummies.head()

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
movie_data = pd.concat([metadata_df[["id"]] , genre_dummies], axis=1)
movie_data

,id,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,...,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,862,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8844,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15602,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31357,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11862,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,949,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,11860,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,45325,0,0,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,9091,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,710,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
links_df = pd.read_csv(links_URL)
links_df = links_df.dropna()
links_df["tmdbId"] = links_df["tmdbId"].map(int)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [21]:
def get_movieId(tmdbId_list, links_df):
    movieid_list = []
    for tmdbId in tmdbId_list:
        if not tmdbId.isdigit():
            movieid_list.append(np.nan)
        else:
            movieId = links_df[links_df["tmdbId"] == int(tmdbId)]["movieId"].tolist()
            if len(movieId) == 1:
                movieid_list.append(movieId[0])
            else:
                movieid_list.append(np.nan)
    return movieid_list

In [22]:
tmdbid_list = movie_data["id"].tolist()
movieid_list = get_movieId(tmdbid_list, links_df)
movie_data["movieId"] = movieid_list
movie_data.head()

,id,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,...,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie,movieId
0,862,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,8844,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
2,15602,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,31357,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,4.0
4,11862,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0


In [23]:
# NaNを削除
movie_data = movie_data[movie_data["movieId"] > 0]
movie_data["id"].count()

42962

In [24]:
scores_df = pd.read_csv(scores_URL)[["movieId", "95%_least_plausible"]]
scores_df.head()

,movieId,95%_least_plausible
0,318.0,4.424716
1,159817.0,4.409009
2,858.0,4.333404
3,50.0,4.294587
4,170705.0,4.261164


In [25]:
movie_data =pd.merge(movie_data, scores_df, on="movieId")
movie_data.drop(["id","movieId"] , axis=1, inplace=True)
movie_data.head()

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,...,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie,95%_least_plausible
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.882039
1,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.226958
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.161669
3,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.841556
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.066052


In [26]:
# データの分割（学習データとテストデータ分ける）
from sklearn.model_selection import train_test_split

# モデル
from sklearn import linear_model

# モデルのインスタンス
l_model = linear_model.LinearRegression()
 
# 説明変数に "price" 以外を利用
X = movie_data.drop("95%_least_plausible", axis=1)

# 目的変数
Y = movie_data["95%_least_plausible"]

# 学習データとテストデータ分ける
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

# モデルのあてはめ
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ ,clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 
# 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 
print(clf.intercept_)

train: LinearRegression 0.0827957769437
test: LinearRegression 0.0801041123114
    Coefficients             Name
19     -0.146525         TV Movie
15     -0.145967          Foreign
10     -0.133427           Horror
7      -0.015848           Action
2      -0.013090           Family
18      0.052045          Western
12      0.071038  Science Fiction
9       0.078430         Thriller
5       0.086155          Romance
14      0.091610              War
4       0.101197          Fantasy
13      0.108459          Mystery
16      0.116138            Music
3       0.122141        Adventure
1       0.122150           Comedy
8       0.128492            Crime
0       0.182488        Animation
17      0.198859      Documentary
11      0.209293          History
6       0.234537            Drama
2.32122058936


***

ここまではがいあのを完全にコピー

In [68]:
metadata = pd.read_csv(metadata_URL)[['budget', 'id', 'revenue', 'runtime']]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
metadata.budget[19730] = 0
metadata.budget[29503] = 0
metadata.budget[35587] = 0

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [70]:
metadata.budget = pd.to_numeric(metadata.budget)

In [71]:
metadata = metadata[metadata['budget']>0]
metadata = metadata[metadata['revenue']>0]
metadata = metadata[metadata['runtime']>0]

In [72]:
tmdbid_list = metadata["id"].tolist()
movieid_list = get_movieId(tmdbid_list, links_df)
metadata["movieId"] = movieid_list
metadata.head()

,budget,id,revenue,runtime,movieId
0,30000000,862,373554033.0,81.0,1.0
1,65000000,8844,262797249.0,104.0,2.0
3,16000000,31357,81452156.0,127.0,4.0
5,60000000,949,187436818.0,170.0,6.0
8,35000000,9091,64350171.0,106.0,9.0


In [73]:
metadata =  metadata[metadata["movieId"] > 0]
metadata["id"].count()

5357

In [74]:
metadata =pd.merge(metadata, scores_df, on="movieId")
metadata.drop(["id","movieId"] , axis=1, inplace=True)
metadata.head()

,budget,revenue,runtime,95%_least_plausible
0,30000000,373554033.0,81.0,3.882039
1,65000000,262797249.0,104.0,3.226958
2,16000000,81452156.0,127.0,2.841556
3,60000000,187436818.0,170.0,3.832650
4,35000000,64350171.0,106.0,2.983756


In [75]:
# データの分割（学習データとテストデータ分ける）
from sklearn.model_selection import train_test_split

# モデル
from sklearn import linear_model

# モデルのインスタンス
l_model = linear_model.LinearRegression()
 
# 説明変数に "price" 以外を利用
X = metadata.drop("95%_least_plausible", axis=1)

# 目的変数
Y = metadata["95%_least_plausible"]

# 学習データとテストデータ分ける
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

# モデルのあてはめ
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ ,clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 
# 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 
print(clf.intercept_)

train: LinearRegression 0.11455269848
test: LinearRegression 0.106363238076
   Coefficients     Name
0 -2.596164e-09   budget
1  8.534381e-10  revenue
2  6.759755e-03  runtime
2.30887040763


In [77]:
metadata_df = pd.read_csv(metadata_URL)[["id", "genres"]]
metadata_df["genres"] = metadata_df["genres"].map(make_list_from_str)
metadata_df["genres"] = metadata_df["genres"].map(extract_names)
metadata_df["genre_count"] = metadata_df["genres"].map(lambda x: len(x))
genres_df = pd.read_csv(genres_URL)
all_genre_id = genres_df["id"].tolist()
all_genre_name = genres_df["name"].tolist()
metadata_df = metadata_df[metadata_df["genre_count"] > 0]
genre_dummies = pd.get_dummies(metadata_df["genres"].apply(pd.Series).stack()).sum(level=0)
genre_dummies = genre_dummies[all_genre_name]
movie_data = pd.concat([metadata_df[["id"]] , genre_dummies], axis=1)
tmdbid_list = movie_data["id"].tolist()
movieid_list = get_movieId(tmdbid_list, links_df)
movie_data["movieId"] = movieid_list
movie_data = movie_data[movie_data["movieId"] > 0]
movie_data.head()

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,...,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie,movieId
0,862,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,8844,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
2,15602,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,31357,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,4.0
4,11862,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0


In [79]:
metadata = pd.read_csv(metadata_URL)[['budget', 'id', 'revenue', 'runtime']]
metadata.budget[19730] = 0
metadata.budget[29503] = 0
metadata.budget[35587] = 0
metadata.budget = pd.to_numeric(metadata.budget)
metadata = metadata[metadata['budget']>0]
metadata = metadata[metadata['revenue']>0]
metadata = metadata[metadata['runtime']>0]
tmdbid_list = metadata["id"].tolist()
movieid_list = get_movieId(tmdbid_list, links_df)
metadata["movieId"] = movieid_list
metadata.head()
metadata =  metadata[metadata["movieId"] > 0]
metadata.head()

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,budget,id,revenue,runtime,movieId
0,30000000,862,373554033.0,81.0,1.0
1,65000000,8844,262797249.0,104.0,2.0
3,16000000,31357,81452156.0,127.0,4.0
5,60000000,949,187436818.0,170.0,6.0
8,35000000,9091,64350171.0,106.0,9.0


In [84]:
merged = pd.merge(movie_data, metadata, on='movieId')
merged = pd.merge(merged, scores_df, on='movieId')
merged.drop(["id_x","id_y", "movieId"] , axis=1, inplace=True)
merged

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,...,War,Foreign,Music,Documentary,Western,TV Movie,budget,revenue,runtime,95%_least_plausible
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,30000000,373554033.0,81.0,3.882039
1,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,65000000,262797249.0,104.0,3.226958
2,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,16000000,81452156.0,127.0,2.841556
3,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,60000000,187436818.0,170.0,3.832650
4,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,35000000,64350171.0,106.0,2.983756
5,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,58000000,352194034.0,130.0,3.423798
6,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,62000000,107879496.0,106.0,3.649377
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,44000000,13681765.0,192.0,3.410690
8,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,98000000,10017322.0,119.0,2.694739
9,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,52000000,116112375.0,178.0,3.785011


### 目的変数はratingスコア、説明変数は予算、興行収入、上映時間をくわえた

In [85]:
# データの分割（学習データとテストデータ分ける）
from sklearn.model_selection import train_test_split

# モデル
from sklearn import linear_model

# モデルのインスタンス
l_model = linear_model.LinearRegression()
 
# 説明変数に "price" 以外を利用
X = merged.drop("95%_least_plausible", axis=1)

# 目的変数
Y = merged["95%_least_plausible"]

# 学習データとテストデータ分ける
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

# モデルのあてはめ
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ ,clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 
# 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 
print(clf.intercept_)

train: LinearRegression 0.201998219022
test: LinearRegression 0.214548445573
    Coefficients             Name
15 -7.938138e-01          Foreign
10 -2.127594e-01           Horror
7  -1.651048e-01           Action
2  -1.607588e-01           Family
19 -8.062623e-02         TV Movie
1  -5.442705e-02           Comedy
16 -4.687441e-02            Music
5  -3.141176e-02          Romance
4  -1.622615e-02          Fantasy
9  -6.249431e-03         Thriller
20 -2.098975e-09           budget
21  9.734993e-10          revenue
3   1.284215e-03        Adventure
22  4.774270e-03          runtime
14  3.810700e-02              War
11  4.119863e-02          History
12  8.940953e-02  Science Fiction
13  1.082160e-01          Mystery
8   1.266655e-01            Crime
6   1.575125e-01            Drama
0   1.780475e-01        Animation
18  2.646165e-01          Western
17  3.169014e-01      Documentary
2.50632836292


### 説明変数を興行収入にして、重回帰分析


In [86]:
# データの分割（学習データとテストデータ分ける）
from sklearn.model_selection import train_test_split

# モデル
from sklearn import linear_model

# モデルのインスタンス
l_model = linear_model.LinearRegression()
 
# 説明変数に "price" 以外を利用
X = merged.drop(["95%_least_plausible", "revenue"], axis=1)

# 目的変数
Y = merged["revenue"]

# 学習データとテストデータ分ける
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5,random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

# モデルのあてはめ
clf = l_model.fit(X_train,y_train)
print("train:",clf.__class__.__name__ ,clf.score(X_train,y_train))
print("test:",clf.__class__.__name__ , clf.score(X_test,y_test))
 
# 偏回帰係数
print(pd.DataFrame({"Name":X.columns,
                    "Coefficients":clf.coef_}).sort_values(by='Coefficients') )
 
# 切片 
print(clf.intercept_)

train: LinearRegression 0.547097222639
test: LinearRegression 0.55171525053
    Coefficients             Name
11 -4.186356e+07          History
18 -2.781296e+07          Western
15 -2.006806e+07          Foreign
6  -1.541197e+07            Drama
7  -1.207149e+07           Action
16 -1.116661e+07            Music
14 -1.083537e+07              War
13 -8.647997e+06          Mystery
12 -7.506687e+06  Science Fiction
9  -6.833033e+06         Thriller
8  -4.848498e+06            Crime
1  -4.043272e+06           Comedy
20  2.884988e+00           budget
21  7.713494e+05          runtime
4   7.072818e+06          Fantasy
5   1.039719e+07          Romance
17  1.231812e+07      Documentary
2   1.358423e+07           Family
10  1.405370e+07           Horror
0   2.561281e+07        Animation
3   2.687958e+07        Adventure
19  2.945461e+07         TV Movie
-74490573.7353


さきほどの結果と比較するとおもしろい。TV Movie, Horror, Familyなどはratingには負に効いていたのに、興行収入では正に効いている。逆にWestern, Dramaなどは反対の挙動を示してる。